# Libraries

In [438]:
import pandas as pd
from getpass import getuser
from collections import defaultdict

# Load and inspect dataset

In [439]:
# Get the current user's name
user = getuser()

# Path to the dataset
data_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\fifa.csv'

# Read the dataset with a different encoding
df = pd.read_csv(data_path, encoding='ISO-8859-1')

# Extract relevant columns

In [440]:
# First, let's extract the year from the 'tournament_id' column and create a new column 'year'
df['year'] = df['tournament_id'].str.extract(r'WC-(\d{4})').astype(int)

# Now, filter the data for tournaments after 1986
filtered_df = df[df['year'] > 2020]

# Extract relevant columns for goal events and match results
goals_df = filtered_df[['tournament_name', 'group_name','match_name', 'match_id', 'player_team_name','match_date', 'minute_regulation', 
               'team_id', 'own_goal']]

# Remove all observations where group_name is "not applicable"
goals_df = goals_df[goals_df['group_name'] != 'not applicable']

# Convert 'match_date' to datetime format
goals_df['match_date'] = pd.to_datetime(goals_df['match_date'], format='%m/%d/%Y')

# Sort the dataset by 'match_date' in ascending order (oldest first) and 'minute_regulation'
goals_df = goals_df.sort_values(by=['match_date', 'minute_regulation'], ascending=[True, True])

# Display the first few rows to confirm the sorting
goals_df.head()


,tournament_name,group_name,match_name,match_id,player_team_name,match_date,minute_regulation,team_id,own_goal
2548,2022 FIFA World Cup,Group A,Qatar v Ecuador,M-2022-01,Ecuador,2022-11-20,16,T-24,0
2549,2022 FIFA World Cup,Group A,Qatar v Ecuador,M-2022-01,Ecuador,2022-11-20,31,T-24,0
2550,2022 FIFA World Cup,Group B,England v Iran,M-2022-02,England,2022-11-21,35,T-27,0
2560,2022 FIFA World Cup,Group B,United States v Wales,M-2022-04,United States,2022-11-21,36,T-80,0
2551,2022 FIFA World Cup,Group B,England v Iran,M-2022-02,England,2022-11-21,43,T-27,0


# Recreate Leauge Table after first two matchdays

### Filter out the matches from the last match day in the goals_df DataFrame.

In [441]:
# Step 1: Find the last match date for each tournament and group
last_dates = goals_df.groupby(['tournament_name', 'group_name'])['match_date'].max().reset_index()

# Step 2: Create the dataset with all games excluding the last match day (goals_before_last_day)
goals_before_last_day = goals_df.merge(last_dates, on=['tournament_name', 'group_name', 'match_date'], how='outer', indicator=True)
goals_before_last_day = goals_before_last_day[goals_before_last_day['_merge'] == 'left_only'].drop(columns=['_merge'])

# Step 3: Create the dataset with only the last match day games (goals_last_day)
goals_last_day = goals_df.merge(last_dates, on=['tournament_name', 'group_name', 'match_date'])

# Step 4: Split match_name into home and away teams and remove match_name column for both datasets

# For goals_before_last_day
goals_before_last_day[['home', 'away']] = goals_before_last_day['match_name'].str.split(' v ', expand=True)
goals_before_last_day = goals_before_last_day.drop(columns=['match_name'])

# For goals_last_day
goals_last_day[['home', 'away']] = goals_last_day['match_name'].str.split(' v ', expand=True)
goals_last_day = goals_last_day.drop(columns=['match_name'])


## create df  of aggregate data for each match

In [442]:
# Initialize an empty list to store the results
results = []

# Iterate over each match in the dataset
for match_id, group in goals_before_last_day.groupby(['tournament_name', 'group_name', 'match_id', 'home', 'away']):
    # Initialize goals_home and goals_away for each match
    goals_home = 0
    goals_away = 0
    
    # Loop through each row in the group
    for _, row in group.iterrows():
        # Check if the team is the home team and no own goal
        if row['home'] == row['player_team_name'] and row['own_goal'] == 0:
            goals_home += 1
        # If the team is the away team and no own goal, increment goals_away
        elif row['away'] == row['player_team_name'] and row['own_goal'] == 0:
            goals_away += 1

    # Append the results to the list, keeping home and away columns
    results.append({
        'tournament_name': match_id[0],
        'group_name': match_id[1],
        'match_id': match_id[2],
        'home': match_id[3],
        'away': match_id[4],
        'goals_home': goals_home,
        'goals_away': goals_away
    })

# Convert the list into a DataFrame
agg_goals_before_last_day = pd.DataFrame(results)

# Add a new column 'won' based on the comparison of goals_home and goals_away
agg_goals_before_last_day['won'] = agg_goals_before_last_day.apply(
    lambda row: 1 if row['goals_home'] > row['goals_away'] else (-1 if row['goals_home'] < row['goals_away'] else 0), 
    axis=1
)

# Display the updated DataFrame (use print if outside of Jupyter)
display(agg_goals_before_last_day)



,tournament_name,group_name,match_id,home,away,goals_home,goals_away,won
0,2022 FIFA World Cup,Group A,M-2022-01,Qatar,Ecuador,0,2,-1
1,2022 FIFA World Cup,Group A,M-2022-03,Senegal,Netherlands,0,2,-1
2,2022 FIFA World Cup,Group A,M-2022-18,Qatar,Senegal,1,3,-1
3,2022 FIFA World Cup,Group A,M-2022-19,Netherlands,Ecuador,1,1,0
4,2022 FIFA World Cup,Group B,M-2022-02,England,Iran,6,2,1
5,2022 FIFA World Cup,Group B,M-2022-04,United States,Wales,1,1,0
6,2022 FIFA World Cup,Group B,M-2022-17,Wales,Iran,0,2,-1
7,2022 FIFA World Cup,Group C,M-2022-05,Argentina,Saudi Arabia,1,2,-1
8,2022 FIFA World Cup,Group C,M-2022-22,Poland,Saudi Arabia,2,0,1
9,2022 FIFA World Cup,Group C,M-2022-24,Argentina,Mexico,2,0,1


## aggregate data for home and away games

In [443]:
# Step 1: Home games aggregation with match count
home_games = agg_goals_before_last_day.groupby(['tournament_name', 'group_name', 'home']).agg(
    goals_scored=('goals_home', 'sum'),
    goals_conceded=('goals_away', 'sum'),
    points_home=('won', lambda x: sum(3 if result == 1 else (1 if result == 0 else 0) for result in x)),
    match_count_home=('match_id', 'count')  # Count how many match_id for each team at home
).reset_index()

# Step 2: Away games aggregation with match count
away_games = agg_goals_before_last_day.groupby(['tournament_name', 'group_name', 'away']).agg(
    goals_scored=('goals_away', 'sum'),
    goals_conceded=('goals_home', 'sum'),
    points_away=('won', lambda x: sum(3 if result == -1 else (1 if result == 0 else 0) for result in x)),
    match_count_away=('match_id', 'count')  # Count how many match_id for each team at away
).reset_index()


## aggregate data after first two matches

In [444]:
# Step 1: Merge home_games and away_games on tournament_name, group_name, and home with away
all_games_before_last = pd.merge(
    home_games,
    away_games,
    left_on=['tournament_name', 'group_name', 'home'],
    right_on=['tournament_name', 'group_name', 'away'],
    how='outer',
    suffixes=('_home', '_away')
)

# Step 2: For teams that only appear in home_games, assign home to team and copy relevant columns
all_games_before_last['team'] = all_games_before_last['home'].fillna(all_games_before_last['away'])

# Step 3: For goals_scored, goals_conceded, and points, handle missing values
all_games_before_last['goals_scored'] = all_games_before_last['goals_scored_home'].fillna(0) + all_games_before_last['goals_scored_away'].fillna(0)
all_games_before_last['goals_conceded'] = all_games_before_last['goals_conceded_home'].fillna(0) + all_games_before_last['goals_conceded_away'].fillna(0)
all_games_before_last['points'] = all_games_before_last['points_home'].fillna(0) + all_games_before_last['points_away'].fillna(0)

# Step 4: Add goals_difference column
all_games_before_last['goals_difference'] = all_games_before_last['goals_scored'] - all_games_before_last['goals_conceded']

# Step 5: Sum match_count_home and match_count_away to get total match count for each team
all_games_before_last['total_matches'] = all_games_before_last['match_count_home'].fillna(0) + all_games_before_last['match_count_away'].fillna(0)

# Step 6: Adjust points if total_matches is 1 (indicating a likely 0-0 draw)
# If total_matches == 1, assume the second match ended 0-0, so add 1 point
all_games_before_last.loc[all_games_before_last['total_matches'] == 1, 'points'] += 1

# Step 7: Drop unnecessary columns used in the merge process
all_games_before_last = all_games_before_last[['tournament_name', 'group_name', 'team', 'goals_scored', 'goals_conceded', 'points', 'goals_difference', 'total_matches']]

# Step 8: Initial sort by points, goals_difference, and goals_scored within each tournament and group
all_games_before_last = all_games_before_last.sort_values(
    by=['tournament_name', 'group_name', 'points', 'goals_difference', 'goals_scored'],
    ascending=[True, True, False, False, False]
).reset_index(drop=True)

# Step 9: Initialize a new column for tie-break results, which will store either the tie-break result or 'no need'
all_games_before_last['tiebreaker'] = 'no need'

# Step 10: Identify rows with tied points, goals_difference, and goals_scored and apply the tie-breaker
def check_tiebreaker(row1, row2, agg_data):
    """
    Check the tie-breaker based on the head-to-head match result from agg_goals_before_last_day.
    Return the team that won (if any) or 'tie'.
    """
    # Look for the head-to-head match between the two teams in agg_goals_before_last_day
    match = agg_data[((agg_data['home'] == row1['team']) & (agg_data['away'] == row2['team'])) |
                     ((agg_data['home'] == row2['team']) & (agg_data['away'] == row1['team']))]
    
    if not match.empty:
        if match.iloc[0]['won'] == 1:
            return row1['team']  # Home team won
        elif match.iloc[0]['won'] == -1:
            return row2['team']  # Away team won
        else:
            return 'tie'  # It's a draw
    else:
        return 'no result'  # No match found

# Step 11: Loop through the sorted dataframe and apply the tie-breaker if needed
for i in range(len(all_games_before_last) - 1):
    row1 = all_games_before_last.iloc[i]
    row2 = all_games_before_last.iloc[i + 1]
    
    # Check if the two rows have identical values for points, goal difference, and goals scored
    if (row1['points'] == row2['points'] and
        row1['goals_difference'] == row2['goals_difference'] and
        row1['goals_scored'] == row2['goals_scored']):
        
        # Apply the tie-breaker by checking the head-to-head match result
        tiebreak_result = check_tiebreaker(row1, row2, agg_goals_before_last_day)
        
        # Store the tie-breaker result for both teams involved
        all_games_before_last.at[i, 'tiebreaker'] = tiebreak_result
        all_games_before_last.at[i + 1, 'tiebreaker'] = tiebreak_result

# Step 12: Add group standing by ranking teams within each group based on points, goal difference, and goals scored
all_games_before_last['standing'] = all_games_before_last.groupby(['tournament_name', 'group_name']).apply(
    lambda x: x.sort_values(by=['points', 'goals_difference', 'goals_scored'], ascending=[False, False, False])
).reset_index(drop=True).groupby(['tournament_name', 'group_name']).cumcount() + 1


C:\Users\aldi\AppData\Local\Temp\ipykernel_19080\1100631210.py:79: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_games_before_last['standing'] = all_games_before_last.groupby(['tournament_name', 'group_name']).apply(


# Recreate League Table after last match day

In [445]:
# Step 1: Sort goals_last_day by minute_regulation in ascending order, grouped by tournament_name and group_name
goals_last_day_sorted = goals_last_day.sort_values(by=['tournament_name', 'group_name', 'match_id', 'minute_regulation'], ascending=True)

# Step 2: Create a new DataFrame to track overall and last match goals for each team
goals_tracking = pd.DataFrame(columns=['tournament_name', 'group_name', 'team', 
                                       'before_last_game_goals_scored', 'before_last_game_goals_conceded', 
                                       'last_game_goals_scored', 'last_game_goals_conceded', 
                                       'total_goal_difference', 'total_goals_scored', 'total_goals_conceded',
                                       'before_last_game_points', 'last_game_points'])

# Step 3: Initialize the goals_tracking table by adding the teams and their points before the last match
for _, row in all_games_before_last.iterrows():
    # Retrieve points and other stats from all_games_before_last and rename the variables accordingly
    new_row = pd.DataFrame({
        'tournament_name': [row['tournament_name']],
        'group_name': [row['group_name']],
        'team': [row['team']],
        'before_last_game_goals_scored': [int(row['goals_scored'])],  # Goals scored before last match
        'before_last_game_goals_conceded': [int(row['goals_conceded'])],  # Goals conceded before last match
        'before_last_game_stading': [int(row['standing'])],  # Standing before last game
        'last_game_goals_scored': [0],  # Initialize with 0 for the last match
        'last_game_goals_conceded': [0],  # Initialize with 0 for the last match
        'total_goal_difference': [int(row['goals_scored']) - int(row['goals_conceded'])],  # Calculate initial goal difference
        'total_goals_scored': [0],  # Initialize total goals scored column
        'total_goals_conceded': [0],  # Initialize total goals conceded column
        'before_last_game_points': [int(row['points'])],  # Use points from all_games_before_last
        'last_game_points': [0]  # Initialize last game points with 0
    })
    goals_tracking = pd.concat([goals_tracking, new_row], ignore_index=True)

# Step 4: Iterate through the sorted and filtered last match goals and update the goals_tracking table
for _, goal in goals_last_day_sorted.iterrows():
    tournament_name = goal['tournament_name']
    group_name = goal['group_name']
    home_team = goal['home']
    away_team = goal['away']
    player_team = goal['player_team_name']
    own_goal = goal['own_goal']

    # Ensure that we are processing only goals from the same tournament and group
    if tournament_name in goals_tracking['tournament_name'].values and group_name in goals_tracking['group_name'].values:
        # Update the goals based on who scored the goal (own goal or normal goal)
        if own_goal == 0:  # Normal goal
            if player_team == home_team:
                # Home team scored, update home scored and away conceded
                goals_tracking.loc[(goals_tracking['team'] == home_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_game_goals_scored'] += 1
                goals_tracking.loc[(goals_tracking['team'] == away_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_game_goals_conceded'] += 1
            elif player_team == away_team:
                # Away team scored, update away scored and home conceded
                goals_tracking.loc[(goals_tracking['team'] == away_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_game_goals_scored'] += 1
                goals_tracking.loc[(goals_tracking['team'] == home_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_game_goals_conceded'] += 1
        elif own_goal == 1:  # Own goal
            if player_team == home_team:
                # Own goal by home team, away team scores
                goals_tracking.loc[(goals_tracking['team'] == away_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_game_goals_scored'] += 1
                goals_tracking.loc[(goals_tracking['team'] == home_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_game_goals_conceded'] += 1
            elif player_team == away_team:
                # Own goal by away team, home team scores
                goals_tracking.loc[(goals_tracking['team'] == home_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_game_goals_scored'] += 1
                goals_tracking.loc[(goals_tracking['team'] == away_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_game_goals_conceded'] += 1

# Step 5: Update total_goal_difference and add total goals scored and conceded
goals_tracking['total_goal_difference'] = (goals_tracking['before_last_game_goals_scored'] + 
                                           goals_tracking['last_game_goals_scored']).astype(int) - \
                                          (goals_tracking['before_last_game_goals_conceded'] + 
                                           goals_tracking['last_game_goals_conceded']).astype(int)

goals_tracking['total_goals_scored'] = (goals_tracking['before_last_game_goals_scored'] + goals_tracking['last_game_goals_scored']).astype(int)
goals_tracking['total_goals_conceded'] = (goals_tracking['before_last_game_goals_conceded'] + goals_tracking['last_game_goals_conceded']).astype(int)

# Step 6: Compare last_game_goals_scored with last_game_goals_conceded and assign points to last_game_points
for _, row in goals_tracking.iterrows():
    if row['last_game_goals_scored'] > row['last_game_goals_conceded']:
        goals_tracking.loc[goals_tracking['team'] == row['team'], 'last_game_points'] = 3  # Win
    elif row['last_game_goals_scored'] == row['last_game_goals_conceded']:
        goals_tracking.loc[goals_tracking['team'] == row['team'], 'last_game_points'] = 1  # Draw
    else:
        goals_tracking.loc[goals_tracking['team'] == row['team'], 'last_game_points'] = 0  # Loss
# Step 7: Calculate the total points by adding before_last_game_points and last_game_points
goals_tracking['total_points'] = goals_tracking['before_last_game_points'] + goals_tracking['last_game_points']

In [446]:
# Step 1: Sort goals_last_day by minute_regulation in ascending order, grouped by tournament_name and group_name
goals_last_day_sorted = goals_last_day.sort_values(by=['tournament_name', 'group_name', 'match_id', 'minute_regulation'], ascending=True)

# Step 2: Create a new DataFrame to track overall and last match goals for each team
goals_tracking = pd.DataFrame(columns=['tournament_name', 'group_name', 'team', 
                                       'before_last_game_goals_scored', 'before_last_game_goals_conceded', 
                                       'last_game_goals_scored', 'last_game_goals_conceded', 
                                       'total_goal_difference', 'total_goals_scored', 'total_goals_conceded',
                                       'before_last_game_points', 'last_game_points', 'total_points', 'last_game_standing',
                                       'temporary_standing', 'changes', '1st', '2nd', '3rd', '4th'])

# Step 3: Initialize the goals_tracking table by adding the teams and their points before the last match
for _, row in all_games_before_last.iterrows():
    new_row = pd.DataFrame({
        'tournament_name': [row['tournament_name']],
        'group_name': [row['group_name']],
        'team': [row['team']],
        'before_last_game_goals_scored': [int(row['goals_scored'])],  # Goals scored before last match
        'before_last_game_goals_conceded': [int(row['goals_conceded'])],  # Goals conceded before last match
        'last_game_goals_scored': [0],  # Initialize with 0 for the last match
        'last_game_goals_conceded': [0],  # Initialize with 0 for the last match
        'total_goal_difference': [int(row['goals_scored']) - int(row['goals_conceded'])],  # Calculate initial goal difference
        'total_goals_scored': [0],  # Initialize total goals scored column
        'total_goals_conceded': [0],  # Initialize total goals conceded column
        'before_last_game_points': [int(row['points'])],  # Use points from all_games_before_last
        'last_game_points': [0],  # Initialize last game points with 0
        'total_points': [int(row['points'])],  # Initialize total points with before last match points
        'last_game_standing': [int(row['standing'])],  # Initial standing
        'temporary_standing': [int(row['standing'])],  # Temporary standing for comparison
        'changes': [0],  # Initialize changes with 0
        '1st': [0], '2nd': [0], '3rd': [0], '4th': [0]  # Initialize standing counters
    })
    goals_tracking = pd.concat([goals_tracking, new_row], ignore_index=True)

# Step 4: Iterate through the sorted and filtered last match goals and update the goals_tracking table
for _, goal in goals_last_day_sorted.iterrows():  # All steps after step 4 are within this loop now
    tournament_name = goal['tournament_name']
    group_name = goal['group_name']
    match_id = goal['match_id']
    home_team = goal['home']
    away_team = goal['away']
    player_team = goal['player_team_name']
    own_goal = goal['own_goal']

    # Ensure that we are processing only goals from the same tournament and group
    if tournament_name in goals_tracking['tournament_name'].values and group_name in goals_tracking['group_name'].values:
        # Update the goals based on who scored the goal (own goal or normal goal)
        if own_goal == 0:  # Normal goal
            if player_team == home_team:
                # Home team scored, update home scored and away conceded
                goals_tracking.loc[(goals_tracking['team'] == home_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_game_goals_scored'] += 1
                goals_tracking.loc[(goals_tracking['team'] == away_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_game_goals_conceded'] += 1
            elif player_team == away_team:
                # Away team scored, update away scored and home conceded
                goals_tracking.loc[(goals_tracking['team'] == away_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_game_goals_scored'] += 1
                goals_tracking.loc[(goals_tracking['team'] == home_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_game_goals_conceded'] += 1
        elif own_goal == 1:  # Own goal
            if player_team == home_team:
                # Own goal by home team, away team scores
                goals_tracking.loc[(goals_tracking['team'] == away_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_game_goals_scored'] += 1
                goals_tracking.loc[(goals_tracking['team'] == home_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_game_goals_conceded'] += 1
            elif player_team == away_team:
                # Own goal by away team, home team scores
                goals_tracking.loc[(goals_tracking['team'] == home_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_game_goals_scored'] += 1
                goals_tracking.loc[(goals_tracking['team'] == away_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_game_goals_conceded'] += 1

    # Step 5: Update total_goal_difference and add total goals scored and conceded
    goals_tracking['total_goal_difference'] = (goals_tracking['before_last_game_goals_scored'] + 
                                               goals_tracking['last_game_goals_scored']).astype(int) - \
                                              (goals_tracking['before_last_game_goals_conceded'] + 
                                               goals_tracking['last_game_goals_conceded']).astype(int)

    goals_tracking['total_goals_scored'] = (goals_tracking['before_last_game_goals_scored'] + goals_tracking['last_game_goals_scored']).astype(int)
    goals_tracking['total_goals_conceded'] = (goals_tracking['before_last_game_goals_conceded'] + goals_tracking['last_game_goals_conceded']).astype(int)

    # Step 6: Compare last_game_goals_scored with last_game_goals_conceded and assign points to last_game_points
    for _, row in goals_tracking.iterrows():
        if row['last_game_goals_scored'] > row['last_game_goals_conceded']:
            goals_tracking.loc[goals_tracking['team'] == row['team'], 'last_game_points'] = 3  # Win
        elif row['last_game_goals_scored'] == row['last_game_goals_conceded']:
            goals_tracking.loc[goals_tracking['team'] == row['team'], 'last_game_points'] = 1  # Draw
        else:
            goals_tracking.loc[goals_tracking['team'] == row['team'], 'last_game_points'] = 0  # Loss

    # Step 7: Calculate the total points by adding before_last_game_points and last_game_points
    goals_tracking['total_points'] = goals_tracking['before_last_game_points'] + goals_tracking['last_game_points']

    # Step 8: Sort the goals_tracking table by total_points, total_goal_difference, and total_goals_scored
    goals_tracking = goals_tracking.groupby(['tournament_name', 'group_name']).apply(
        lambda x: x.sort_values(by=['total_points', 'total_goal_difference', 'total_goals_scored'],
                                ascending=[False, False, False])
    ).reset_index(drop=True)

    # Step 9: Assign the new standing based on the updated sorting
    goals_tracking['last_game_standing'] = goals_tracking.groupby(['tournament_name', 'group_name']).cumcount() + 1

    



C:\Users\aldi\AppData\Local\Temp\ipykernel_19080\2926684592.py:105: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  goals_tracking = goals_tracking.groupby(['tournament_name', 'group_name']).apply(
C:\Users\aldi\AppData\Local\Temp\ipykernel_19080\2926684592.py:105: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  goals_tracking = goals_tracking.groupby(['tournament_name', 'group_name']).apply(
C:\Users\aldi\AppD

In [ ]:
# Step 10: Track changes and update standing counters
    for i, row in goals_tracking.iterrows():
        # Debugging: Print the group names for comparison
        print(f"Row group_name: {row['group_name']}, goals_tracking group_name at index {i}: {goals_tracking['group_name'][i]}")
        
        if (row['tournament_name'] == goals_tracking['tournament_name'][i]) and (row['group_name'] == goals_tracking['group_name'][i]):
            if row['temporary_standing'] != row['last_game_standing']:
                # Increment changes and update temporary_standing
                goals_tracking.loc[goals_tracking['team'] == row['team'], 'changes'] += 1
                goals_tracking.loc[goals_tracking['team'] == row['team'], 'temporary_standing'] = row['last_game_standing']

            # Correctly increment the standing counter after each goal and print the updates
            if row['last_game_standing'] == 1:
                goals_tracking.loc[(goals_tracking['team'] == row['team']) &
                                (goals_tracking['tournament_name'] == row['tournament_name']) &
                                (goals_tracking['group_name'] == row['group_name']), '1st'] += 1
            elif row['last_game_standing'] == 2:
                goals_tracking.loc[(goals_tracking['team'] == row['team']) &
                                (goals_tracking['tournament_name'] == row['tournament_name']) &
                                (goals_tracking['group_name'] == row['group_name']), '2nd'] += 1
            elif row['last_game_standing'] == 3:
                goals_tracking.loc[(goals_tracking['team'] == row['team']) &
                                (goals_tracking['tournament_name'] == row['tournament_name']) &
                                (goals_tracking['group_name'] == row['group_name']), '3rd'] += 1
            elif row['last_game_standing'] == 4:
                goals_tracking.loc[(goals_tracking['team'] == row['team']) &
                                (goals_tracking['tournament_name'] == row['tournament_name']) &
                                (goals_tracking['group_name'] == row['group_name']), '4th'] += 1

    # Display the final results of the standings
    display(goals_tracking[['team', '1st', '2nd', '3rd', '4th', 'changes']])